In [1]:
from gpp_scripts.sem_label.contrastive.make_dataset import *
from timer import Timer

In [2]:
from pathlib import Path

TEMP_DIR = Path("/var/tmp/bvu-iswc25-e03-runtime")
TEMP_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
from libactor.cache import IdentObj
from experiments.config import DATA_DIR, LIBACTOR_STORAGE_DIR
from experiments.dag import create_data_actor
from libactor.storage import GlobalStorage

# GlobalStorage.init(LIBACTOR_STORAGE_DIR)
# uncomment this to test the runtime
GlobalStorage.init(TEMP_DIR / "libactor")
data_actor = create_data_actor()
kgdb = data_actor.db_actor.kgdbs[KGName.Wikidata]
ident_kgdb = IdentObj(kgdb.args.get_key(), kgdb)

2025-05-14 05:32:10.197 | INFO     | libactor.storage._global_storage:init:41 - GlobalStorage: /var/tmp/bvu-iswc25-e03-runtime/libactor


In [4]:
len(data_actor.forward("wt250").value), len(data_actor.forward("t2dv2").value)

2025-05-14 05:32:12.487 | DEBUG    | sm_datasets.datasets:_fix_redirections:271 - Redirect entity: Q153382 to None
2025-05-14 05:32:12.663 | DEBUG    | sm_datasets.datasets:_fix_redirections:271 - Redirect entity: Q2402815 to Q108749732
2025-05-14 05:32:12.940 | DEBUG    | sm_datasets.datasets:_fix_redirections:271 - Redirect entity: Q4671431 to Q19819292
2025-05-14 05:32:13.211 | DEBUG    | sm_datasets.datasets:_fix_redirections:271 - Redirect entity: Q65088885 to Q56342425
2025-05-14 05:32:13.286 | DEBUG    | sm_datasets.datasets:_fix_redirections:271 - Redirect entity: Q6594748 to Q101531124
2025-05-14 05:32:13.288 | DEBUG    | sm_datasets.datasets:_fix_redirections:271 - Redirect entity: Q11794421 to Q24062643
2025-05-14 05:32:13.353 | DEBUG    | sm_datasets.datasets:_fix_redirections:271 - Redirect entity: Q22097387 to Q3417643
2025-05-14 05:32:13.410 | DEBUG    | sm_datasets.datasets:_fix_redirections:271 - Redirect entity: Q370015 to Q17545162
2025-05-14 05:32:13.414 | DEBUG    

(250, 224)

In [5]:
with Timer().watch_and_report("make WT250 dataset"):
    dataset = "wt250"
    exs, ds = make_v220_dataset(
        dataset,
        # DATA_DIR / f"experiments/sem-label-data-v210/{dataset}.pkl",
        TEMP_DIR / f"experiments/sem-label-data-v210/{dataset}.pkl",
    )
with Timer().watch_and_report("ensure embedding"):
    ensure_embedding(ds, "BAAI/bge-m3")

2025-05-14 05:32:32.297 | DEBUG    | timer:watch_and_report:74 - DataActor.load_dataset deserialize: 0.167 seconds


processing:   0%|          | 0/250 [00:00<?, ?it/s]

Load embedding model: ...
Load embedding model: 3.134 seconds


In [5]:
serde.pickle.ser(exs, DATA_DIR / f"experiments/sem-model-data/{dataset}.pkl")

In [4]:
df = ds["target_labels"]
df

,is_prop,label,aliases,description,similar_labels
id,,,,,
P102,True,member of political party,"[party, member of, member of party, party memb...",the political party of which a person is or ha...,"{'P118': 2, 'P131': 1, 'P1344': 1, 'P17': 2, '..."
P106,True,occupation,"[profession, job, work, career, employment, cr...","occupation of a person; see also ""field of wor...",{}
P1082,True,population,"[inhabitants, human population, Population, Be...",number of people inhabiting the place; number ...,{}
P1083,True,maximum capacity,"[number of seats, seats, seating capacity, cap...",number of people allowed for a venue or vehicle,{}
P1093,True,gross tonnage,"[ship gross tonnage, GT, tonnage gross, GRT, B...",unitless index related to a ship's overall int...,{}
...,...,...,...,...,...
Q8719053,False,music venue,"[concert venue, live music venue, Konzerthalle...",any location used for a concert or musical per...,{}
Q902104,False,private university,"[private college, Private Universität, centro ...",higher education institution not operated by a...,{}
Q920182,False,main battle tank,"[MBT, battle tank, universal tank, main battle...",tank designed to conduct primary combat missio...,{}


manually fix description in wrong language

In [6]:
from kgdata.models import MultiLingualString

kgdb = data_actor.get_kgdb("t2dv2")
cls = kgdb.ontology.value.classes["http://dbpedia.org/ontology/AcademicJournal"]
cls.description = MultiLingualString.en(
    "peer-reviewed periodical relating to an academic discipline"
)

cls = kgdb.ontology.value.classes["http://dbpedia.org/ontology/Airline"]
cls.description = MultiLingualString.en(
    "company that provides air transport services for traveling passengers and freight"
)

cls = kgdb.ontology.value.classes["http://dbpedia.org/ontology/BaseballPlayer"]
cls.description = MultiLingualString.en("person who plays games of baseball")

In [ ]:
with Timer().watch_and_report("make WT250 dataset"):
    dataset = "wt250"
    exs, ds = make_v220_dataset(
        dataset,
        # DATA_DIR / f"experiments/sem-label-data-v210/{dataset}.pkl",
        TEMP_DIR / f"experiments/sem-label-data-v210/{dataset}.pkl",
    )
with Timer().watch_and_report("ensure embedding"):
    ensure_embedding(ds, "BAAI/bge-m3")

In [7]:
with Timer().watch_and_report("make t2dv2 dataset"):
    dataset = "t2dv2"
    exs, ds = make_v220_dataset(
        dataset,
        # DATA_DIR / f"experiments/sem-label-data-v210/{dataset}.pkl"
        TEMP_DIR / f"experiments/sem-label-data-v210/{dataset}.pkl",
    )

2025-05-14 05:35:16.621 | DEBUG    | timer:watch_and_report:74 - DataActor.load_dataset deserialize: 0.032 seconds
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#gYear, Converter=<function parse_xsd_gyear at 0x126d0f380>
Traceback (most recent call last):
  File "/Users/rook/workspace/.venv/lib/python3.12/site-packages/rdflib/term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "/Users/rook/workspace/.venv/lib/python3.12/site-packages/rdflib/xsd_datetime.py", line 624, in parse_xsd_gyear
    return date(y, 1, 1)
           ^^^^^^^^^^^^^
ValueError: year -322 is out of range
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#gYear, Converter=<function parse_xsd_gyear at 0x126d0f380>
Traceback (most recent call last):
  File "/Users/rook/workspace/.venv/lib/python3.12/site-packages/rdflib/term.py", line 2163, in _castLexi

make t2dv2 dataset: 2.981 seconds


In [8]:
with Timer().watch_and_report("ensure embedding"):
    ensure_embedding(ds, "BAAI/bge-m3")

processing:   0%|          | 0/224 [00:00<?, ?it/s]

ensure embedding: 57.270 seconds


In [14]:
df = ds["target_labels"]
df

,is_prop,label,aliases,description,similar_labels
id,,,,,
http://dbpedia.org/ontology/AcademicJournal,False,academic journal,[],peer-reviewed periodical relating to an academ...,{}
http://dbpedia.org/ontology/AdministrativeRegion,False,administrative region,[],A PopulatedPlace under the jurisdiction of an ...,{}
http://dbpedia.org/ontology/Airline,False,airline,[],company that provides air transport services f...,{}
http://dbpedia.org/ontology/Airport,False,airport,[],,{}
http://dbpedia.org/ontology/Animal,False,animal,[],,{}
...,...,...,...,...,...
http://dbpedia.org/ontology/usingCountry,True,using country,[],,{}
http://dbpedia.org/ontology/usk,True,approved rating of the Entertainment Software ...,[],,{}
http://dbpedia.org/ontology/water,True,water,[],,{}


In [15]:
serde.pickle.ser(exs, DATA_DIR / f"experiments/sem-model-data/{dataset}.pkl")

In [16]:
dataset = Path(
    "/Users/rook/workspace/projects/resm-v2/data/datasets/wiki-20230620/auto-label/wt-limited-easy-sp51/wt-limited-easy-sp51-v1"
)

In [17]:
exs, ds = make_v220_dataset(
    dataset, DATA_DIR / f"experiments/sem-label-data-v210/{dataset.name}.pkl"
)

In [18]:
serde.pickle.ser(exs, DATA_DIR / f"experiments/sem-model-data/{dataset.name}.pkl")

In [5]:
ensure_embedding(ds, "BAAI/bge-m3")

processing:   0%|          | 0/20290 [00:00<?, ?it/s]

Load embedding model: ...
Load embedding model: 3.225 seconds


In [19]:
for name in ["t2dv2", "wt250"]:
    ds = serde.pickle.deser(DATA_DIR / f"experiments/sem-label-data-v210/{name}.pkl")
    ensure_embedding(ds, "BAAI/bge-m3")

processing:   0%|          | 0/224 [00:00<?, ?it/s]

Load embedding model: ...
Load embedding model: 3.852 seconds


processing:   0%|          | 0/250 [00:00<?, ?it/s]

In [4]:
exs = load_dataset_from_disk(dataset, kgdb, fix_redirection=False)

In [17]:
def has_edge(sm, rel_uri: str) -> bool:
    for e in sm.edges():
        if e.rel_uri.endswith(rel_uri):
            return True
    return False


for ex in exs:
    if has_edge(ex.sms[0], ":P37"):
        print(ex.id)
        display(ex.table.table.df)
        ex.sms[0].print(env="notebook")
        break

https://en.wikipedia.org/wiki/List_of_submissions_to_the_31st_Academy_Awards_for_Best_Foreign_Language_Film?table_no=0


,Submitting country,Film title used in nomination,Original title,Language(s),Director(s),Result
0,Denmark,Dollars from the Sky,Guld og grønne skove,Danish,Gabriel Axel,Not nominated
1,France,My Uncle,Mon Oncle,French,Jacques Tati,Won Academy Award
2,West Germany,Arms and the Man,Helden,German,Franz Peter Wirth,Nominated
3,India,Madhumati,मधुमती (Madhumati),Hindi,Bimal Roy,Not nominated
4,Italy,Big Deal on Madonna Street,I soliti ignoti,Italian,Mario Monicelli,Nominated
5,Japan,Ballad of Narayama,楢山節考 (Narayama Bushikō),Japanese,Keisuke Kinoshita,Not nominated
6,Spain,La Venganza,La Venganza,Spanish,Juan Antonio Bardem,Nominated
7,Sweden,The Magician,Ansiktet,Swedish,Ingmar Bergman,Not nominated
8,United Arab Republic,Cairo Station,باب الحديد (Bab el hadid),Egyptian Arabic,Youssef Chahine,Not nominated
9,Yugoslavia,The Road a Year Long,La strada lunga un anno,Italian,Giuseppe De Santis,Nominated


HTML(value='<pre>\n00.\t<span style="background: #b7eb8f; color: black; padding: 2px; border-radius: 3px;">[4]…

In [6]:
schema = Schema.from_ontology(kgdb.ontology.value, exs)

In [ ]:
"P37" in schema.props

True

In [13]:
kgdb.pydb.entities["Q142"].props["P37"][0].value.as_entity_id()

'Q150'

In [16]:
any(
    enttypes.get(range, far_distance) <= max_distance
    for range in kgdb.pydb.props["P37"].ranges
)

False

In [8]:
ds["samples"]

[{'table': <sm.inputs.table.ColumnBasedTable at 0x332eb4d40>,
  'context': Context(page_title='2021–22 Thai League 3 Northeastern Region', page_url='https://en.wikipedia.org/wiki/2021%E2%80%9322_Thai_League_3_Northeastern_Region', entities=['Q109569056'], literals=[], content_hierarchy=[<rsoup.core.ContentHierarchy object at 0x332eb84b0>, <rsoup.core.ContentHierarchy object at 0x332eb8510>, <rsoup.core.ContentHierarchy object at 0x332eb8570>]),
  'column_index': [0],
  'column_type': [['Q476028']]},
 {'table': <sm.inputs.table.ColumnBasedTable at 0x332eb2b40>,
  'context': Context(page_title='List of managers at the FIFA World Cup', page_url='https://en.wikipedia.org/wiki/List_of_managers_at_the_FIFA_World_Cup', entities=['Q65078555'], literals=[], content_hierarchy=[<rsoup.core.ContentHierarchy object at 0x332eb9ef0>, <rsoup.core.ContentHierarchy object at 0x332eb9f50>]),
  'column_index': [1],
  'column_type': [['Q5']]},
 {'table': <sm.inputs.table.ColumnBasedTable at 0x332ebde80>,
 

In [5]:
ds.keys()

dict_keys(['samples', 'target_labels', 'target_label_examples'])

In [3]:
from kgdata.wikidata.config import WikidataDirCfg


WikidataDirCfg.init(DATA_DIR / "kgdata/wikidata/20240320")

2025-05-08 00:52:50.890 | INFO     | kgdata.wikidata.config:init:136 - Wikidata directory: /Users/rook/workspace/projects/resm-v2/data/kgdata/wikidata/20240320


In [4]:
missing_class_ids = [
    "Q108392111",
    "Q112099",
    "Q13107770",
    "Q13580678",
    "Q24243801",
    "Q3241045",
    "Q5119",
    "Q561068",
    "Q6644778",
    "Q67608976",
    "Q76514543",
    "Q828803",
    "Q85790993",
]

In [5]:
from gpp.sem_label.feats import GetExamplesArgs, get_examples

get_examples(
    missing_class_ids,
    kgdb.pydb.entity_metadata.cache(),
    kgdb.ontology.value,
    args=GetExamplesArgs(
        manual_example_dir=DATA_DIR / "manual-examples", only_manual_examples=False
    ),
)

2025-05-08 00:52:51.101 | DEBUG    | kgdata.spark.common:get_spark_context:107 - Spark Options: []
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/rook/workspace/.venv/lib/python3.12/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.